In [14]:
import wget #En lugar de usar el user-agent, usé otra librería llamada requests
import re
import requests #Otra forma de descargar páginas

import io
import glob
from bs4 import BeautifulSoup
import os
from datetime import datetime

In [9]:
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from  IPython.display import clear_output

In [2]:
#url_pagina = "https://presidente.gob.mx/secciones/version-estenografica/page/1/"
#wget.download(url_pagina, out="pagina_i.txt")

In [3]:
#url = "https://lopezobrador.org.mx/transcripciones/"
#url = "https://presidente.gob.mx/secciones/version-estenografica/page/2/" 
#r = requests.get(url, allow_redirects=False)
#open('pagina_i.txt', 'wb').write(r.content)

37294

In [4]:
#archivo = open("./pagina_i.txt","r",encoding = "utf-8").read()
#regex = "(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w.,@?^=%&:/~+#-])?"
#re.findall(regex , archivo)[0]

('https', 'lopezobrador.org.mx', '/feed/')

### Páginas con links de las mañaneras en version estenografica

In [4]:
#url = "https://lopezobrador.org.mx/transcripciones/page/" 
url = "https://presidente.gob.mx/secciones/version-estenografica/page/"
paginas = 117  
for i in range(paginas):
    headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'} # https://stackoverflow.com/questions/10606133/sending-user-agent-using-requests-library-in-python
    session = requests.Session()
    retry = Retry(connect=3, backoff_factor=0.5)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)

    r = session.get(url+str(i+1)+"/", headers=headers)
    open('./paginas/pagina_'+str(i+1)+'.txt', 'wb').write(r.content)
    print("pagina "+str(i+1)+" guardada\n")
    clear_output(wait=True)

pagina 117 guardada



### Links de cada mañanera

In [4]:
urls_conf = []

for f_pagina in glob.glob("./paginas/*"):
    #print(f_pagina)
    archivo = open(f_pagina,"r",encoding = "utf-8").read()
    regex = "(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w.,@?^=%&:/~+#-])?"

    urls_conf += [path for protocol,domain,path in 
                  re.findall(regex , archivo)
                  if "version-estenografica" in path
                  ]

In [5]:
print(len(set(urls_conf)))
print(urls_conf[0])

1852
/secciones/version-estenografica/feed/


### Descargar cada mañanera

In [10]:
urls_conf_set = list(set(urls_conf))

for i, path in enumerate(urls_conf_set):
    headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}

    session = requests.Session()
    retry = Retry(connect=3, backoff_factor=0.5)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    url = "https://presidente.gob.mx/"+path
    r = session.get(url, headers=headers)
    try:  
        open("./estenograficas/"+path.replace("/","-")+".txt", 'wb').write(r.content)
        print("pagina "+str(i+1)+" guardada de "+str(len(urls_conf_set)))
        clear_output(wait=True)
    except FileNotFoundError:
        print("no se puede descargar\n")

pagina 1852 guardada de 1852


### Limpiar cada mañanera versión estenografica

In [11]:
for f_pagina in glob.glob("./estenograficas/*"):
    soup = BeautifulSoup(open(f_pagina,"r",encoding="utf-8").read(),'html.parser')
    try: 
        open("./estenograficas_limpias/"+f_pagina.replace("./estenograficas\\",""), "w", encoding="utf-8").write(soup.get_text())
    except FileNotFoundError:
        print("error al guardar\n")

error al guardar

error al guardar

error al guardar



c:\Users\Ulises\anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


### Estandarizar la fecha de cada una de la conferencia de prensa matutina

In [15]:
def normaliza(str_fecha):
    datetimeobject = datetime.strptime(str_fecha[:6] + "20" + str_fecha[6:], '%d.%m.%Y')
    newformat = datetimeobject.strftime("%Y-%m-%d")
    return newformat

In [16]:
fechas_conferencia = []
cnt = 0
for f_pagina in glob.glob("./estenograficas_limpias/*"):
    texto = open(f_pagina,"r",encoding="utf-8").read()
    #fechas_en_archivo = re.findall('[a-zA-Z]+\s+\d{1,2},\s+\d{4}',texto) #La página tiene diferente formato de fechas
    fechas_en_archivo = re.findall('[0-9][0-9]\.[0-9][0-9]\.[0-9][0-9]',texto)
    print(fechas_en_archivo)

    if len(fechas_en_archivo) == 4:
        
        if not os.path.isfile("./estenograficas_limpias_por_fecha/"+normaliza(fechas_en_archivo[0])):
            try: 
                open("./estenograficas_limpias_por_fecha/"+normaliza(fechas_en_archivo[0]),"w",encoding="utf-8").write(texto)
                fechas_conferencia += [normaliza(fechas_en_archivo[0])]
                cnt+=1
            except:
                print("error al guardar\n")
        elif not os.path.isfile("./estenograficas_limpias_por_fecha/"+normaliza(fechas_en_archivo[0])+"_2da"):
            try: 
                open("./estenograficas_limpias_por_fecha/"+normaliza(fechas_en_archivo[0])+"_2da","w",encoding="utf-8").write(texto)
                fechas_conferencia += [normaliza(fechas_en_archivo[0])]
                cnt+=1
                print("Segunda", f_pagina)
            except:
                print("error al guardar\n")
        elif not os.path.isfile("./estenograficas_limpias_por_fecha/"+normaliza(fechas_en_archivo[0])+"_3ra"):
            try: 
                open("./estenograficas_limpias_por_fecha/"+normaliza(fechas_en_archivo[0])+"_3ra","w",encoding="utf-8").write(texto)
                fechas_conferencia += [normaliza(fechas_en_archivo[0])]
                cnt+=1
                print("Tercera", f_pagina)
            except:
                print("error al guardar\n")
    else:
        print("RARO", f_pagina)

['01.02.20', '01.02.20', '01.02.20', '01.02.20']
['01.02.20', '01.02.20', '01.02.20', '01.02.20']
Segunda ./estenograficas_limpias\-01-02-20-version-estenografica-programas-integrales-de-bienestar-y-tren-maya-desde-campeche-campeche-.txt
['01.02.22', '01.02.22', '01.02.22', '01.02.22']
['01.02.23', '01.02.23', '01.02.23', '01.02.23']
['01.03.21', '01.03.21', '01.03.21', '01.03.21']
['01.03.22', '01.03.22', '01.03.22', '01.03.22']
['01.04.20', '01.04.20', '01.04.20', '01.04.20']
['01.04.20', '01.04.20', '01.04.20', '01.04.20']
Segunda ./estenograficas_limpias\-01-04-20-version-estenografica-inauguracion-del-hospital-rural-del-imss-bienestar-en-tlaxiaco-oaxaca-.txt
['01.04.22', '01.04.22', '01.04.22', '01.04.22']
['01.05.20', '01.05.20', '01.05.20', '01.05.20']
[]
RARO ./estenograficas_limpias\-01-05-22-version-estenografica-dia-del-trabajo-en-la-refineria-dos-bocas-olmeca-desde-paraiso-tabasco-.txt
['01.06.20', '01.06.20', '01.06.20', '01.06.20']
['01.06.20', '01.06.20', '01.06.20', '01

In [17]:
cnt

1711